In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/My Drive/hafta12')
!pwd

/content/drive/My Drive/hafta12


In [ ]:
#Gerekli Kütüphaneler
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import itertools
from itertools import chain
from sklearn.feature_selection import RFE
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import VotingClassifier
import warnings
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.figure_factory as ff
from matplotlib.patches import FancyArrowPatch
from sklearn.preprocessing import LabelEncoder
warnings.filterwarnings('ignore')

In [ ]:
data = pd.read_csv('datasetDVM.csv')
data.head()

In [ ]:
#Unnamed: 32 coloumn Nan değeri olduğu için o sütunu siliyoruz.İleride sorun çıkartacaktır.
data = data.drop(['Unnamed: 32','id'],axis = 1)

label encoding

In [ ]:
#Hedef Atamaları yapılmıştır.M=1 B=0 Kötü huylu=1,iyi huylu=0
data.diagnosis.replace(to_replace = dict(M = 1, B = 0), inplace = True)

In [ ]:
data.head()

In [ ]:
#seaborn version ile grafiğe dökme

palette ={0 : 'pink', 1 : 'purple'}
edgecolor = 'grey'

# Plot +
fig = plt.figure(figsize=(12,12))

plt.subplot(221)
ax1 = sns.scatterplot(x = data['radius_worst'], y = data['perimeter_worst'], hue = "diagnosis",
                    data = data, palette = palette, edgecolor=edgecolor)
plt.title('radius_worst vs perimeter_worst')
plt.subplot(222)
ax2 = sns.scatterplot(x = data['area_worst'], y = data['area_mean'], hue = "diagnosis",
                    data = data, palette =palette, edgecolor=edgecolor)
plt.title('area_worst vs area_mean')
plt.subplot(223)
ax3 = sns.scatterplot(x = data['texture_mean'], y = data['texture_worst'], hue = "diagnosis",
                    data = data, palette =palette, edgecolor=edgecolor)
plt.title('texture mean vs texture worst')
plt.subplot(224)
ax4 = sns.scatterplot(x = data['compactness_mean'], y = data['concavity_mean'], hue = "diagnosis",
                    data = data, palette =palette, edgecolor=edgecolor)
plt.title('compactness_mean vs concavity_mean')

fig.suptitle('Positive correlated features', fontsize = 20)
plt.savefig('1')
plt.show()

In [ ]:
# seaborn version :
fig = plt.figure(figsize=(12,12))

plt.subplot(221)
ax1 = sns.scatterplot(x = data['smoothness_mean'], y = data['texture_mean'], hue = "diagnosis",
                    data = data, palette =palette, edgecolor=edgecolor)
plt.title('smoothness mean vs texture mean')
plt.subplot(222)
ax2 = sns.scatterplot(x = data['radius_mean'], y = data['fractal_dimension_worst'], hue = "diagnosis",
                    data = data, palette =palette, edgecolor=edgecolor)
plt.title('radius mean vs fractal dimension_worst')
plt.subplot(223)
ax3 = sns.scatterplot(x = data['texture_mean'], y = data['symmetry_mean'], hue = "diagnosis",
                    data = data, palette =palette, edgecolor=edgecolor)
plt.title('texture mean vs symmetry mean')
plt.subplot(224)
ax4 = sns.scatterplot(x = data['texture_mean'], y = data['symmetry_se'], hue = "diagnosis",
                    data = data, palette =palette, edgecolor=edgecolor)
plt.title('texture mean vs symmetry se')

fig.suptitle('Uncorrelated features', fontsize = 20)
plt.savefig('2')
plt.show()

In [ ]:
#Her Ml alogritması için gerekli kütüphaneler satırlarda import edilcektir.
## SVC sınıfını import ettik
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.metrics import precision_score, recall_score, confusion_matrix, roc_curve, precision_recall_curve, accuracy_score
from sklearn.model_selection import cross_val_score,learning_curve,train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.pipeline import Pipeline,make_pipeline
from matplotlib.cm import rainbow
from sklearn import metrics
sns.set(style = 'darkgrid')
%matplotlib inline

In [ ]:
X = data.iloc[:,1:].values
y = data['diagnosis'].values
print(f"'X' shape: {X.shape}")
print(f"'y' shape: {y.shape}")

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=22,stratify=y)

In [ ]:
#ÖZELLİK ÖLÇEKLENDİRME
scaler = StandardScaler()#stndrt nesnesi üret
X_train = scaler.fit_transform(X_train)
X_test= scaler.transform(X_test)

In [ ]:
# SVC sınıfından bir nesne ürettik
# Parametre olarak farklı kernel trick tipleri verilebilir.
# Başarı oranının değiştiği gözlemlenecektir. ( ‘linear’, ‘poly’, ‘rbf’)
from sklearn.svm import SVC
svc_scores = []
kernels = ['linear', 'poly', 'rbf', 'sigmoid']  #4 çekirdek fonksiyon sırayla uygulandı

#her bir çekirdeği uyguladık
for i in range(len(kernels)):
    svc_classifier = SVC(kernel = kernels[i])
    pipe = make_pipeline(scaler,svc_classifier)
    svc_scores.append(cross_val_score(pipe, X_train, y_train, cv=5, scoring='accuracy').mean())

colors = rainbow(np.linspace(0, 1, len(kernels)))
plt.figure(figsize=(10,7))
plt.bar(kernels, svc_scores, color = colors)
for i in range(len(kernels)):
    plt.text(i, svc_scores[i], svc_scores[i])
plt.xlabel('Kernels')
plt.ylabel('Scores')
plt.title('Support Vector Classifier scores for different kernels')

In [ ]:
print('Accuracy score on Train data: {}'.format(svc_scores[2]*100))
from sklearn import metrics
svc_classifier = SVC(kernel = 'rbf')  #'rbf', 'poly'
pipe = make_pipeline(scaler,svc_classifier)
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
print('Accuracy score on Test data: {}'.format(metrics.accuracy_score(y_test,y_pred)*100))